## 0. Import neccessary packages.

In [1]:
import Recovery_RL_Agent
import env_robot
import numpy as np

In [2]:
robot = env_robot.Env(dim=2)
agent = Recovery_RL_Agent.Agent()

## 1. Recording demonstration

In [3]:
goal_array = np.array(([100,300],[100,500],[300,500],[300,300],[300,200],[300,100]))
demo_act_dict = agent.demo_record(goal_array)
demo_act_dict

OrderedDict([('0', array([100, 300])),
             ('1', array([100, 500])),
             ('2', array([300, 500])),
             ('3', array([300, 300])),
             ('4', array([300, 200])),
             ('5', array([300, 100]))])

## 2. Executing the demonstration and record experience tuples.

In [4]:
repeat_times = 100

for i in range(0,repeat_times):
    # executing the demo action and restore experience tuples in agent
    episode_record = robot.execute_demo_act(demo_act_dict)
    agent.exp_record(episode_record)
    # Reset env, back to start point
    robot.reset()


env_robot.py:48: RuntimeWarning: divide by zero encountered in divide
  return distance/ee_speed


### Note: you can check the experience buffer and its tuples.

In [5]:
exp_tuple_test = agent.get_exp_list()

In [6]:
print(exp_tuple_test[0].state,
exp_tuple_test[0].contact,
exp_tuple_test[0].action,
exp_tuple_test[0].reward,
exp_tuple_test[0].next_state,
exp_tuple_test[0].next_contact,
exp_tuple_test[0].done)

([0, 0], array([0, 0, 0, 0]), {'0': array([100, 300])}, -407.1921103566489, array([100.55892287, 298.64902846]), array([1, 1, 1, 1]), False)


In [7]:
for i in exp_tuple_test:
    print(i.contact)

[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[1 1 1 1]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[2 2 2 2]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[3 3 3 3]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
[2 2 2 2]
[4 4 4 4]
[5 5 5 5]
[0 0 0 0]
[1 1 1 1]
